IMPORTING LIBRARIES

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Dense,Layer,Flatten,Input,Conv2D,MaxPooling2D,BatchNormalization,ReLU
from tensorflow.keras.losses import binary_crossentropy 
from tensorflow.keras.models import Sequential
import keras
import matplotlib.pyplot as plt

MOUNTING THE DRIVE TO IMPORT THE DATA 

In [ ]:
from google.colab import drive
drive.mount ('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


MAKING MODEL FOR PREDICATION 

In [ ]:
model=Sequential([

tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(300,300,3)),
tf.keras.layers.MaxPooling2D(2,2),

tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(300,300,3)),
tf.keras.layers.MaxPooling2D(2,2),

tf.keras.layers.Conv2D(64,(3,3),activation='relu',input_shape=(300,300,3)),
tf.keras.layers.MaxPooling2D(2,2),

tf.keras.layers.Conv2D(64,(3,3),activation='relu',input_shape=(300,300,3)),
tf.keras.layers.MaxPooling2D(2,2),

tf.keras.layers.Conv2D(64,(3,3),activation='relu',input_shape=(300,300,3)),
tf.keras.layers.MaxPooling2D(2,2),

tf.keras.layers.Flatten(),

tf.keras.layers.Dense(152,activation='relu'),

tf.keras.layers.Dense(1,activation='sigmoid')
])

MODEL's SUMMARY

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 149, 149, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 73, 73, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 35, 35, 64)      

COMPILING THE MODEL

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

RESIZING THE IMAGE 

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_data_gen = ImageDataGenerator( rescale=1.0/255. )
validation_datagen = ImageDataGenerator( rescale=1.0/255. )

IMPORTING THE DATA

In [ ]:
train_path = '/content/drive/MyDrive/horse-or-human/train'
validation_path = '/content/drive/MyDrive/horse-or-human/validation'

DIVIDING THE DATA IN TRAIN AND VALIDATION

In [ ]:
train_data_generator = train_data_gen.flow_from_directory(
    train_path,
    target_size= (300,300),
    batch_size=128,
    class_mode = 'binary'
)

Found 1037 images belonging to 2 classes.


In [ ]:
validation_data_generator = validation_datagen.flow_from_directory(
    validation_path,
    target_size = (300,300),
    batch_size = 128,
    class_mode = 'binary'
)

Found 256 images belonging to 2 classes.


MAKING CALLBACK FUNCTION

In [ ]:
class mycallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if(logs.get('accuracy')>0.98):
            print("Please stop training Model ready")
            self.model.stop_training=True

In [ ]:
callback = mycallback()

IMPORTING THE DATA INTO MODEL FOR TRAINING AND PREDICTION

In [ ]:
history = model.fit_generator(
    train_data_generator,

    epochs = 30,
    validation_data = validation_data_generator,
    callbacks=[callback]
    
)

<ipython-input-56-f8e78c4d3693>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/30
9/9 [==============================] - 113s 12s/step - loss: 0.6686 - accuracy: 0.6075 - val_loss: 0.5990 - val_accuracy: 0.6836
Epoch 2/30
9/9 [==============================] - 114s 12s/step - loss: 0.4728 - accuracy: 0.8177 - val_loss: 1.0367 - val_accuracy: 0.6875
Epoch 3/30
9/9 [==============================] - 108s 12s/step - loss: 0.2625 - accuracy: 0.8910 - val_loss: 1.2416 - val_accuracy: 0.8203
Epoch 4/30
9/9 [==============================] - 108s 12s/step - loss: 0.1656 - accuracy: 0.9392 - val_loss: 1.4714 - val_accuracy: 0.8203
Epoch 5/30
9/9 [==============================] - 108s 12s/step - loss: 0.1421 - accuracy: 0.9412 - val_loss: 1.6233 - val_accuracy: 0.8164
Epoch 6/30
9/9 [==============================] - 113s 12s/step - loss: 0.1029 - accuracy: 0.9605 - val_loss: 1.5195 - val_accuracy: 0.8164
Epoch 7/30
9/9 [==============================] - 112s 12s/step - loss: 0.0726 - accuracy: 0.9720 - val_loss: 1.5126 - val_accuracy: 0.8281
Epoch 8/30
9/9 [====

UPLOADING NEW IMAGE AND PREDICTING WETHER IMAGE IS HUMAN OR HORSE

In [ ]:
from google.colab import files
from tensorflow.keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
  path = '/content/'+ fn
  img=image.load_img(path, target_size=(300,300))
  x = image.img_to_array(img)
  x = np.expand_dims(x,axis=0)

  images = np.vstack([x])
  classes = model.predict(images,batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + '\nITS A HUMAN')
  else:
    print(fn+ '\nITS A HORSE') 

Saving download (1).jfif to download (1).jfif
1/1 [==============================] - 0s 54ms/step
[1.]
download (1).jfif
ITS A HUMAN
